# _sync_repo

In [ ]:
#|default_exp cmds._sync_repo
#|export_as_func true

In [ ]:
#|hide
import nblite; from nblite import show_doc; nblite.nbl_export()

Environment variable DISABLE_NBLITE_EXPORT is set to True, skipping export.


In [ ]:
#|top_export
from pathlib import Path

from repoyard._utils.bisync_helper import bisync_helper, SyncSetting
from repoyard.config import get_config, StorageType
from repoyard import const

In [ ]:
#|set_func_signature
def sync_repo(
    config_path: Path,
    repo_full_name: str,
    sync_setting: SyncSetting = SyncSetting.BISYNC,
    force: bool = False,
):
    """
    Syncs a repo with its remote.
    
    Args:
        config_path: Path to the repoyard config file.
        repo_full_name: Name of the repo to sync.
        sync_setting: The sync setting to use.
        force: If True, the sync will be forced. Used in the case 
    """
    ...

Set up testing args

In [ ]:
# Set up test environment
import tempfile
tests_working_dir = const.pkg_path.parent / "tmp_tests"
test_folder_path = Path(tempfile.mkdtemp(prefix="sync_repo", dir="/tmp"))
test_folder_path.mkdir(parents=True, exist_ok=True)
symlink_path = tests_working_dir / "_cmds" / "sync_repo"
symlink_path.parent.mkdir(parents=True, exist_ok=True)
if symlink_path.exists() or symlink_path.is_symlink():
    symlink_path.unlink()
symlink_path.symlink_to(test_folder_path, target_is_directory=True) # So that it can be viewed from within the project working directory
data_path = test_folder_path / ".repoyard"

In [ ]:
# Args (1/2)
config_path = test_folder_path / "repoyard_config" / "config.toml"
sync_setting = SyncSetting.BISYNC
force = False

In [ ]:
# Run init
from repoyard.cmds import init_repoyard
from repoyard.cmds import new_repo
init_repoyard(config_path=config_path, data_path=data_path)

# Add a storage location 'my_remote'
import toml
config_dump = toml.load(config_path)
remote_rclone_path = Path(tempfile.mkdtemp(prefix="rclone_remote", dir="/tmp"))
config_dump['storage_locations']['my_remote'] = {
    'storage_type' : "rclone",
    'store_path' : "repoyard",
}
config_path.write_text(toml.dumps(config_dump))

new_repo(config_path=config_path, repo_name="test_repo", storage_location="my_remote")

Using a non-default config path. Please set the environment variable REPOYARD_CONFIG_PATH to the given config path for repoyard to use it. 
Creating config file at: /tmp/sync_repobkrm_1qe/repoyard_config/config.toml
Creating folder: /tmp/sync_repobkrm_1qe/.repoyard
Creating folder: /tmp/sync_repobkrm_1qe/.repoyard/local_store
Creating rclone config file at: /tmp/sync_repobkrm_1qe/repoyard_config/repoyard_rclone.conf
Done!

You can modify the config at: /tmp/sync_repobkrm_1qe/repoyard_config/config.toml
All repoyard data is stored in: /tmp/sync_repobkrm_1qe/.repoyard
Initialized empty Git repository in /private/tmp/sync_repobkrm_1qe/.repoyard/local_store/my_remote/01K9YZXCA8Y5RB6HWMG8XTXG3S__test_repo/data/.git/


'01K9YZXCA8Y5RB6HWMG8XTXG3S__test_repo'

In [ ]:
# Args (2/2)
repo_full_name = list((data_path / "local_store" / "my_remote").glob("*"))[0].name

In [ ]:
# Put an excluded file into the repo data folder to make sure it is not synced
(data_path / "local_store" / "my_remote" / repo_full_name / const.REPO_DATA_REL_PATH / ".venv").mkdir(parents=True, exist_ok=True)
(data_path / "local_store" / "my_remote" / repo_full_name / const.REPO_DATA_REL_PATH / ".venv" / "test.txt").write_text("test");

# Function body

Process args

In [ ]:
#|export
config = get_config(config_path)

In [ ]:
# Set up a rclone remote path for testing
config.rclone_config_path.write_text(f"""
[my_remote]
type = alias
remote = {remote_rclone_path}
""");

Find the repo meta

In [ ]:
#|export
from repoyard._repos import get_repoyard_meta
repoyard_meta = get_repoyard_meta(config)

if repo_full_name not in repoyard_meta.by_full_name:
    raise ValueError(f"Repo '{repo_full_name}' not found.")

repo_meta = repoyard_meta.by_full_name[repo_full_name]

Check if the repo is in a local storage location, in which case quit.

In [ ]:
#|export
if repo_meta.get_storage_location_config(config).storage_type == StorageType.LOCAL:
    pass
    #|return_line

Sync the repometa

In [ ]:
#|export
bisync_helper(
    rclone_config_path=config.rclone_config_path,
    sync_setting=sync_setting,
    local_path=repo_meta.get_local_path(config),
    remote=repo_meta.storage_location,
    remote_path=repo_meta.get_remote_path(config),
    force=force,
    include=[f"/{const.REPO_METAFILE_REL_PATH}"]
)

In [ ]:
# Check that the synced worked
from repoyard._utils import rclone_lsjson
_lsjson = rclone_lsjson(
    rclone_config_path=config.rclone_config_path,
    source=repo_meta.storage_location,
    source_path=repo_meta.get_remote_path(config)
)
assert "repometa.toml" in {f["Name"] for f in _lsjson}

Sync the repoconf

In [ ]:
#|export
bisync_helper(
    rclone_config_path=config.rclone_config_path,
    sync_setting=sync_setting,
    local_path=repo_meta.get_local_repoconf_path(config),
    remote=repo_meta.storage_location,
    remote_path=repo_meta.get_remote_repoconf_path(config),
    force=force,
)

In [ ]:
# Check that the synced worked
from repoyard._utils import rclone_lsjson
_lsjson = rclone_lsjson(
    rclone_config_path=config.rclone_config_path,
    source=repo_meta.storage_location,
    source_path=repo_meta.get_remote_repoconf_path(config)
)
assert ".repoyard_exclude" in {f["Name"] for f in _lsjson}

Get the now locally synced conf files for the sync of the repo data

In [ ]:
#|export
_repoyard_include_path = repo_meta.get_local_repoconf_path(config) / ".repoyard_include"
_repoyard_exclude_path = repo_meta.get_local_repoconf_path(config) / ".repoyard_exclude"
_repoyard_filters_path = repo_meta.get_local_repoconf_path(config) / ".repoyard_filters"

_repoyard_include_path = _repoyard_include_path if _repoyard_include_path.exists() else None
_repoyard_exclude_path = _repoyard_exclude_path if _repoyard_exclude_path.exists() else None
_repoyard_filters_path = _repoyard_filters_path if _repoyard_filters_path.exists() else None

Sync the repo data

In [ ]:
#|export
bisync_helper(
    rclone_config_path=config.rclone_config_path,
    sync_setting=sync_setting,
    local_path=repo_meta.get_local_repodata_path(config),
    remote=repo_meta.storage_location,
    remote_path=repo_meta.get_remote_repodata_path(config),
    force=force,
    include_path=_repoyard_include_path,
    exclude_path=_repoyard_exclude_path,
    filters_path=_repoyard_filters_path,
)

Refresh the repoyard meta file

In [ ]:
#|export
from repoyard._repos import refresh_repoyard_meta
refresh_repoyard_meta(config)

In [ ]:
# Check that the synced worked
from repoyard._utils import rclone_lsjson
_lsjson = rclone_lsjson(
    rclone_config_path=config.rclone_config_path,
    source=repo_meta.storage_location,
    source_path=repo_meta.get_remote_repodata_path(config)
)
assert ".git" in {f["Name"] for f in _lsjson}
assert ".venv" not in {f["Name"] for f in _lsjson}